In [1]:
import pandas as pd
import numpy as np
import json
import hashlib
import collections
from datetime import datetime
from datetime import timedelta
from collections import OrderedDict

In [2]:
dataEvents = pd.read_csv('/Users/manuelgomezmoratilla/Desktop/TFG/data_processing_scripts/data/anonamyze_all_data_collection_v2.csv', sep=";")

In [3]:
pd.options.mode.chained_assignment = None  # default='warn'
def sequenceWithinPuzzles(dataEvents, group = 'all'):
    tutorialList = ['1. One Box', '2. Separated Boxes', '3. Rotate a Pyramid', '4. Match Silhouettes', '5. Removing Objects', '6. Stretch a Ramp', '7. Max 2 Boxes', '8. Combine 2 Ramps', '9. Scaling Round Objects', 'Sandbox']
    #Remove SandBox and tutorial levels.
    dataEvents['group'] = [json.loads(x)['group'] if 'group' in json.loads(x).keys() else '' for x in dataEvents['data']]
    dataEvents['user'] = [json.loads(x)['user'] if 'user' in json.loads(x).keys() else '' for x in dataEvents['data']]
    # removing those rows where we dont have a group and a user that is not guest
    dataEvents = dataEvents[((dataEvents['group'] != '') & (dataEvents['user'] != '') & (dataEvents['user'] != 'guest'))]
    dataEvents['group_user_id'] = dataEvents['group'] + '~' + dataEvents['user']
    # filtering to only take the group passed as argument
    if(group != 'all'):
        dataEvents = dataEvents[dataEvents['group'].isin(group)]
    # Data Cleaning
    dataEvents['time'] = pd.to_datetime(dataEvents['time'])
    dataEvents = dataEvents.sort_values('time') 
    
    newDataEvents = []
    #Select puzzle and actions
    notSelectedEvents = ['ws-mode_change', 'ws-click_nothing', 'ws-click_disabled', 'ws-select_shape', 'ws-deselect_shape', 'ws-paint', 'ws-palette_change', 'ws-toggle_paint_display', 'ws-toggle_snapshot_display', 'ws-create_user', 'ws-login_user']
    #Selected puzzles
    selectedPuzzles = ['Square Cross-Sections', 'Bird Fez', 'Pi Henge', '45-Degree Rotations',  'Pyramids are Strange', 'Boxes Obscure Spheres', 'Object Limits', 'Angled Silhouette',
                    'Sugar Cones','Stranger Shapes', 'Tall and Small', 'Ramp Up and Can It', 'More Than Meets Your Eye', 'Not Bird', 'Zzz', 'Bull Market', 'Orange Dance', 'Bear Market']

    eventsWithMetaData = ['ws-create_shape', 'ws-delete_shape', 'ws-rotate_shape', 'ws-scale_shape', 'ws-move_shape']

    for user in dataEvents['group_user_id'].unique():
            #Select rows
            user_events = dataEvents[dataEvents['group_user_id'] == user]
            user_events_na_dropped = user_events.dropna()
            activePuzzle = None
            nAttempt = 1
            prevCheck = False
            prevEvent = None
            figureDict = dict()
            for enum, event in user_events_na_dropped.iterrows():
                #Ignore event
                if (prevCheck == True):
                    if (event['type'] == 'ws-puzzle_complete'):
                        prevEvent['metadata']['correct'] = True
                        newDataEvents.append(prevEvent)
                        prevCheck = False
                        prevEvent = None
                        continue
                    else:
                        prevEvent['metadata']['correct'] = False
                    newDataEvents.append(prevEvent)
                    prevCheck = False
                    prevEvent = None
                        
                if (event['type'] in notSelectedEvents):
                        continue

                elif(event['type'] == 'ws-start_level'):
                    activePuzzle = json.loads(event['data'])['task_id']
                    event['task_id'] = activePuzzle
                    if (event['task_id'] in selectedPuzzles):
                        event['n_attempt'] = nAttempt
                        event['metadata'] = dict()
                        newDataEvents.append(event)

                elif (event['type'] == 'ws-create_shape'):
                    event['task_id'] = activePuzzle
                    if (event['task_id'] in selectedPuzzles):
                        event['n_attempt'] = nAttempt
                        shape_id = json.loads(event['data'])['objSerialization']
                        shape_type = json.loads(event['data'])['shapeType']
                        figureDict[shape_id] = shape_type
                        event['metadata'] = dict()
                        event['metadata']['shape_id'] = shape_id 
                        event['metadata']['shape_type'] = shape_type 
                        newDataEvents.append(event)

                elif (event['type'] == 'ws-delete_shape' or event['type'] == 'ws-move_shape'):
                    event['task_id'] = activePuzzle
                    if (event['task_id'] in selectedPuzzles):
                        event['n_attempt'] = nAttempt
                        if (event['type'] == 'ws-delete_shape'):
                            idList = json.loads(event['data'])['deletedShapes']
                        elif (event['type'] == 'ws-move_shape'):
                            idList = json.loads(event['data'])['selectedObjects']
                        for shapeId in idList:
                            shape_id = shapeId
                            shape_type = figureDict[shape_id]
                            event['metadata'] = dict()
                            event['metadata']['shape_id'] = shape_id 
                            event['metadata']['shape_type'] = shape_type 
                            newDataEvents.append(event)

                elif (event['type'] == 'ws-rotate_shape' or event['type'] == 'ws-scale_shape'):
                    event['task_id'] = activePuzzle
                    if (event['task_id'] in selectedPuzzles):
                        event['n_attempt'] = nAttempt
                        shape_id = json.loads(event['data'])['selectedObject']
                        shape_type = figureDict[shape_id]
                        event['metadata'] = dict()
                        event['metadata']['shape_id'] = shape_id 
                        event['metadata']['shape_type'] = shape_type 
                        newDataEvents.append(event)

                elif ((event['type'] == 'ws-exit_to_menu') and (activePuzzle in selectedPuzzles)):
                    figureDict.clear()
                    nAttempt +=1
   
                else :
                    event['task_id'] = activePuzzle
                    if (event['task_id'] in selectedPuzzles):
                        event['n_attempt'] = nAttempt
                        event['metadata'] = dict()
                        if (event['type'] == 'ws-check_solution'):
                            prevCheck = True
                            prevEvent = event
                        else:
                            newDataEvents.append(event)

    taskDf = pd.DataFrame(newDataEvents, columns=['id', 'time', 'group_user_id', 'task_id', 'n_attempt', 'type', 'metadata']) 

    data = taskDf
    listEvent = ['ws-rotate_view', 'ws-rotate_shape', 'ws-undo_action', 'ws-move_shape', 'ws-snapshot', 'ws-scale_shape']
    
    dataConvert2 = []
    for user in data['group_user_id'].unique():
        individualDf = data[data['group_user_id'] == user]
        #Current action set
        currentAction = []
        #String with action types
        actionString = ""
        actualEvent = 'None'
        for enum, event in individualDf.iterrows():
            key = event['group_user_id']
            key_split = key.split('~')
            event['group_id'] = key_split[0]
            event['user'] = key_split[1]
            actualEvent = event['type']
            eq = True
            for a in currentAction:
                if (a['type'] != actualEvent):
                    #Ver si podemos compactar
                    eq = False
                    
            if (eq == False):      
                igual = True
                prev = ""
                for a2 in currentAction:
                    if (a2['type'] != prev):
                        if (prev == "") :
                            igual = True
                        else:
                            igual = False
                    prev = a2['type']
                if ((igual == True) and (prev in listEvent)):
                    add = currentAction[0]
                    #add['type'] = add['type'] + 'x' + str(len(currentAction))
                    add['n_times'] = dict()
                    add['n_times'][add['type']] = len(currentAction)
                    dataConvert2.append(add)
                    currentAction.clear()
                    currentAction.append(event)     
                else: #igual != True 
                    for a in currentAction:
                        a['n_times'] = dict()
                        a['n_times'][a['type']] = 1
                        dataConvert2.append(a)
                    currentAction.clear()
                    currentAction.append(event)
            else: #eq = True
                if (event['type'] not in listEvent):
                    currentAction.append(event)
                    for a in currentAction:
                        a['n_times'] = dict()
                        a['n_times'][a['type']] = 1
                        dataConvert2.append(a)
                    currentAction.clear()
                    
                else:
                    if (len(currentAction) > 0):
                            if (currentAction[0]['type'] in eventsWithMetaData):
                                #Event with metadata, check if it is the same shape_id
                                if (currentAction[0]['metadata']['shape_id'] == event['metadata']['shape_id']):
                                    currentAction.append(event)
                                else:
                                    add = currentAction[0]
                                    #add['type'] = add['type'] + 'x' + str(len(currentAction))
                                    add['n_times'] = dict()
                                    add['n_times'][add['type']] = len(currentAction)
                                    dataConvert2.append(add)
                                    currentAction.clear()
                                    currentAction.append(event)
                            #Event without metaData, just concatenate.
                            else:
                                currentAction.append(event) 

                    elif (len(currentAction) == 0):
                        currentAction.append(event)
                
                    
        #Add last elems
        #We must check if last elems can be also replaced.
        final = ""
        if (len(currentAction) > 0):
            igual2 = True
            prev = ""
            for a2 in currentAction:
                if (a2['type'] != prev):
                    if (prev == "") :
                        igual2 = True
                    else:
                        igual2 = False
                prev = a2['type']
            if ((igual == True) and (prev in listEvent)):
                add = currentAction[0]
                #add['type'] = add['type'] + 'x' + str(len(currentAction))
                add['n_times'] = dict()
                add['n_times'][add['type']] = len(currentAction)
                dataConvert2.append(add)
                currentAction.clear()
                currentAction.append(event)     
            else: #igual != True 
                for a in currentAction:
                    a['n_times'] = dict()
                    a['n_times'][a['type']] = 1
                    dataConvert2.append(a)
                currentAction.clear()
                currentAction.append(event)
               
    #Create dataframe from list
    #consecutiveDf = pd.DataFrame(dataConvert2, columns=['id', 'time', 'group_user_id', 'task_id', 'n_attempt', 'type', 'metadata'])         
    data = pd.DataFrame(dataConvert2, columns=['group_id', 'user', 'task_id', 'n_attempt', 'type', 'n_times', 'metadata']) 
    return data

In [4]:
#Segunda fase : compactamos eventos de manipulacion de una misma figura.
def collapseManipulationEvents (dataFrame):
    # ahora vamos a compactar acciones de 3 subacciones
    df = dataFrame
    eventsWithMetaData = ['ws-create_shape', 'ws-delete_shape', 'ws-rotate_shape', 'ws-scale_shape', 'ws-move_shape']
    dataConvert = []
    for user in df['user'].unique():
        individualDf = df[df['user'] == user]
        currentAction = []
        for enum, event in individualDf.iterrows():
            currentAction.append(event)
            if (len(currentAction) == 3):
                #Intentamos etiquetar la accion
                actionString = ""
                for action in currentAction:
                    actionString = actionString + action['type'] + " "
                if ((currentAction[0]['type'] in eventsWithMetaData and currentAction[1]['type'] in eventsWithMetaData and currentAction[2]['type'] in eventsWithMetaData) and (currentAction[0]['type'] != currentAction[1]['type'] and currentAction[1]['type'] != currentAction[2]['type']) and
                    (currentAction[0]['metadata']['shape_id'] == currentAction[1]['metadata']['shape_id'] and currentAction[0]['metadata']['shape_id'] == currentAction[2]['metadata']['shape_id'])) :
                    accion = currentAction[0]
                    nVeces = currentAction[0]['n_times']
                    for action in currentAction[1]['n_times'].keys():
                        nVeces[action] = currentAction[1]['n_times'][action]
                    for action in currentAction[2]['n_times'].keys():
                        if action in nVeces.keys():
                            action2 = action + "(2)"
                            nVeces[action2] = currentAction[2]['n_times'][action]
                        else:
                            nVeces[action] = currentAction[2]['n_times'][action]
                    nameString = 'ws-'
                    sub = currentAction[0]['type'].split("-")
                    action1 = sub[1].split("_")[0]
                    sub = currentAction[1]['type'].split("-")
                    action2 = sub[1].split("_")[0]
                    sub = currentAction[2]['type'].split("-")
                    action3 = sub[1].split("_")[0]
                    nameString = nameString + action1 + "_" + action2 + "_" + action3 +"_shape"
                    accion['type'] = nameString
                    accion['ntimes'] = nVeces
                    dataConvert.append(accion)
                    currentAction.clear()        
                else:
                    elem = currentAction.pop(0)
                    dataConvert.append(elem)
        #Añadir los ultimos elementos.
        for e in currentAction :
            dataConvert.append(e)
                #Limpiamos la lista
                #currentAction.clear()

    newDfConvert = pd.DataFrame(dataConvert, columns=['group_id', 'user', 'task_id', 'n_attempt', 'type', 'n_times', 'metadata'])
     
    # ahora vamos a compactar acciones de 2 subacciones
    dataConvert = []
    for user in newDfConvert['user'].unique():
        individualDf = newDfConvert[newDfConvert['user'] == user]
        currentAction = []
        for enum, event in individualDf.iterrows():
            currentAction.append(event)
            if (len(currentAction) == 2):
                #Intentamos etiquetar la accion
                actionString = ""
                for action in currentAction:
                    actionString = actionString + action['type'] + " "

                if (actionString == ('ws-start_level ws-puzzle_started ')):
                    currentAction.clear()

                elif ((currentAction[0]['type'] in eventsWithMetaData and currentAction[1]['type'] in eventsWithMetaData) and (currentAction[0]['type'] != currentAction[1]['type']) and
                    (currentAction[0]['metadata']['shape_id'] == currentAction[1]['metadata']['shape_id'])) :
                    accion = currentAction[0]
                    nVeces = currentAction[0]['n_times']
                    for action in currentAction[1]['n_times'].keys():
                        nVeces[action] = currentAction[1]['n_times'][action]
                    nameString = 'ws-'
                    sub = currentAction[0]['type'].split("-")
                    action1 = sub[1].split("_")[0]
                    sub = currentAction[1]['type'].split("-")
                    action2 = sub[1].split("_")[0]
                    nameString = nameString + action1 + "_" + action2 + "_shape"
                    accion['type'] = nameString
                    accion['ntimes'] = nVeces
                    dataConvert.append(accion)
                    currentAction.clear()

                else:
                    elem = currentAction.pop(0)
                    dataConvert.append(elem)
        #Añadir los ultimos elementos.
        for e in currentAction :
            dataConvert.append(e)
                #Limpiamos la lista
                #currentAction.clear()

    newDfConvert = pd.DataFrame(dataConvert, columns=['group_id', 'user', 'task_id', 'n_attempt', 'type', 'n_times', 'metadata'])
    
    return newDfConvert           




In [5]:
#Estas funciones son la primera y segunda capa del algoritmo de sequenceWithinPuzzles
df = sequenceWithinPuzzles(dataEvents)
newDfConvert = collapseManipulationEvents(dataFrame = df)
newDfConvert

,group_id,user,task_id,n_attempt,type,n_times,metadata
409376,cb71040b5bd1341a34afc24961536ebd,56ccce25ead834182d605eff319bfa2c,Sugar Cones,1,ws-create_rotate_shape,"{'ws-create_shape': 1, 'ws-rotate_shape': 2}","{'shape_id': 1, 'shape_type': 5}"
409470,cb71040b5bd1341a34afc24961536ebd,56ccce25ead834182d605eff319bfa2c,Sugar Cones,1,ws-create_rotate_move_shape,"{'ws-create_shape': 1, 'ws-rotate_shape': 2, '...","{'shape_id': 2, 'shape_type': 5}"
409720,cb71040b5bd1341a34afc24961536ebd,56ccce25ead834182d605eff319bfa2c,Sugar Cones,1,ws-rotate_view,{'ws-rotate_view': 10},{}
409932,cb71040b5bd1341a34afc24961536ebd,56ccce25ead834182d605eff319bfa2c,Sugar Cones,1,ws-move_shape,{'ws-move_shape': 2},"{'shape_id': 2, 'shape_type': 5}"
410020,cb71040b5bd1341a34afc24961536ebd,56ccce25ead834182d605eff319bfa2c,Sugar Cones,1,ws-move_shape,{'ws-move_shape': 2},"{'shape_id': 1, 'shape_type': 5}"
...,...,...,...,...,...,...,...
396953,e21640b4aea9349ad77d86d6017cb061,eeb4ac2443dd0496ffbefa6780f4d7d4,Bird Fez,1,ws-rotate_view,{'ws-rotate_view': 1},{}
396987,e21640b4aea9349ad77d86d6017cb061,eeb4ac2443dd0496ffbefa6780f4d7d4,Bird Fez,1,ws-move_shape,{'ws-move_shape': 4},"{'shape_id': 3, 'shape_type': 5}"
397091,e21640b4aea9349ad77d86d6017cb061,eeb4ac2443dd0496ffbefa6780f4d7d4,Bird Fez,1,ws-move_shape,{'ws-move_shape': 3},"{'shape_id': 1, 'shape_type': 6}"
397103,e21640b4aea9349ad77d86d6017cb061,eeb4ac2443dd0496ffbefa6780f4d7d4,Bird Fez,1,ws-move_shape,{'ws-move_shape': 1},"{'shape_id': 2, 'shape_type': 1}"


In [6]:
def selectEvents(dataFrame):
    pathSoluciones = "/Users/manuelgomezmoratilla/Downloads/Fall19studylevels-final (1)/"
    data = newDfConvert #Como parametro

    listaArchivos = []
    for i in range(1,10):
            listaArchivos.append("public" + str(i)+".json")
            listaArchivos.append("annie" + str(i)+".json")

    puzzleSol = dict()
    for archivo in listaArchivos:
        nuevaSol = pd.read_json(pathSoluciones + archivo,orient='index')
        puzzleSol[nuevaSol[0]['puzzleName']] = nuevaSol[0]['shapeData']


    #HASTA AQUI TENEMOS EN PUZZLE SOL CADA PUZZLE CON TODA SU SOLUCION.
    #AHORA VAMOS A PONER COMO VALOR DEL DICT SOLO LAS FORMAS QUE CONTIENE SIN TENER EN CUENTA ROTACIONES, ESCALA...
    puzzleSolTypes = dict()
    for puzzle in puzzleSol.keys():
        listaFormas = []
        for shape in puzzleSol[puzzle]:
            listaFormas.append(shape['shapeType'])
        puzzleSolTypes[puzzle] = listaFormas


    errorList = []
    for user in data['user'].unique():
                #Select rows
                user_events = data[data['user'] == user]
                user_events_na_dropped = user_events.dropna()
                for attempt in user_events_na_dropped['n_attempt'].unique():
                    attempt_events = user_events_na_dropped[user_events_na_dropped['n_attempt'] == attempt]
                    shapeList = []
                    for enum, event in attempt_events.iterrows():
                        if ("create" in event['type']):
                            shapeList.append(event['metadata']['shape_type'])
                        if ("delete" in event['type']):
                            if (event['metadata']['shape_type'] in shapeList):
                                shapeList.remove(event['metadata']['shape_type'])
                        elif (event['type'] == 'ws-check_solution'):
                            listaSolucion = puzzleSolTypes[event['task_id']]
                            listaSolucion.sort()
                            shapeList.sort()
                            event['shapeTypes_solution'] = listaSolucion
                            event['shapeTypes_submitted'] = shapeList
                            if (event['metadata']['correct'] == True):
                                if listaSolucion == shapeList:
                                    event['same_Solution'] = True
                                else:
                                    event['same_Solution'] = False
                                errorList.append(event)

    #COMPARAMOS SI LA SOLUCION QUE EL ALUMNO HA PROPUESTO Y LA QUE SE PENSO ES LA MISMA.
    errorDf = pd.DataFrame(errorList, columns=['user', 'task_id', 'shapeTypes_solution', 'shapeTypes_submitted', 'metadata', 'same_Solution'])                     

    #Ahora vamos a seleccionar solo aquellos eventos que pertenezcan a un estudiante que este intentando
    #solucionar el puzzle de manera identica a la solucion propuesta por el diseñador. Para ello creamos un
    #diccionario con los puzzles que cada usuario esta intentando solucionar de esa manera, y después cogemos solo
    #esos puzzles de ese usuario.
    #Es una manera de simplificar el problema dada la complejidad de analizar los errores en puzzles que no tienen 
    #una unica solucion
    identicalDict = {}
    selectedList = ['Square Cross-Sections', 'Bird Fez', 'Pi Henge', '45-Degree Rotations',  'Pyramids are Strange', 'Boxes Obscure Spheres', 'Object Limits', 'Angled Silhouette',
                    'Sugar Cones','Stranger Shapes', 'Tall and Small', 'Ramp Up and Can It', 'More Than Meets Your Eye', 'Not Bird', 'Zzz', 'Bull Market', 'Orange Dance', 'Bear Market']


    for user in errorDf['user'].unique():
        user_events = errorDf[errorDf['user'] == user]
        for enum, event in user_events.iterrows():
            if user not in identicalDict.keys():
                identicalDict[user] = []
            else:
                if event['same_Solution'] == True:
                    lista = identicalDict[user]
                    lista.append(event['task_id'])

    usersSelected = []
    for user in newDfConvert['user'].unique():
        user_events = newDfConvert[newDfConvert['user'] == user]
        if user not in identicalDict.keys():
            continue
        userList = identicalDict[user]
        for enum, event in user_events.iterrows():
            if (event['task_id'] in userList and event['task_id'] in selectedList):
                usersSelected.append(event)

    eventsToUse = pd.DataFrame(usersSelected, columns=['group_id', 'user', 'task_id', 'n_attempt', 'type', 'n_times', 'metadata'])
    return eventsToUse
    


In [7]:
eventsToUse = selectEvents(dataFrame = newDfConvert)
eventsToUse

,group_id,user,task_id,n_attempt,type,n_times,metadata
409376,cb71040b5bd1341a34afc24961536ebd,56ccce25ead834182d605eff319bfa2c,Sugar Cones,1,ws-create_rotate_shape,"{'ws-create_shape': 1, 'ws-rotate_shape': 2}","{'shape_id': 1, 'shape_type': 5}"
409470,cb71040b5bd1341a34afc24961536ebd,56ccce25ead834182d605eff319bfa2c,Sugar Cones,1,ws-create_rotate_move_shape,"{'ws-create_shape': 1, 'ws-rotate_shape': 2, '...","{'shape_id': 2, 'shape_type': 5}"
409720,cb71040b5bd1341a34afc24961536ebd,56ccce25ead834182d605eff319bfa2c,Sugar Cones,1,ws-rotate_view,{'ws-rotate_view': 10},{}
409932,cb71040b5bd1341a34afc24961536ebd,56ccce25ead834182d605eff319bfa2c,Sugar Cones,1,ws-move_shape,{'ws-move_shape': 2},"{'shape_id': 2, 'shape_type': 5}"
410020,cb71040b5bd1341a34afc24961536ebd,56ccce25ead834182d605eff319bfa2c,Sugar Cones,1,ws-move_shape,{'ws-move_shape': 2},"{'shape_id': 1, 'shape_type': 5}"
...,...,...,...,...,...,...,...
395738,e21640b4aea9349ad77d86d6017cb061,7e95e0d124f5d7852af4b05082db7b1c,Sugar Cones,10,ws-rotate_view,{'ws-rotate_view': 2},{}
395768,e21640b4aea9349ad77d86d6017cb061,7e95e0d124f5d7852af4b05082db7b1c,Sugar Cones,10,ws-check_solution,{'ws-check_solution': 1},{'correct': False}
395791,e21640b4aea9349ad77d86d6017cb061,7e95e0d124f5d7852af4b05082db7b1c,Sugar Cones,10,ws-rotate_view,{'ws-rotate_view': 2},{}
395845,e21640b4aea9349ad77d86d6017cb061,7e95e0d124f5d7852af4b05082db7b1c,Sugar Cones,10,ws-move_shape,{'ws-move_shape': 1},"{'shape_id': 1, 'shape_type': 5}"


In [8]:
def commonErrors(dataFrame) :
    
    pathSoluciones = "/Users/manuelgomezmoratilla/Downloads/Fall19studylevels-final (1)/"
    data = dataFrame #Como parametro

    listaArchivos = []
    for i in range(1,10):
            listaArchivos.append("public" + str(i)+".json")
            listaArchivos.append("annie" + str(i)+".json")

    puzzleSol = dict()
    for archivo in listaArchivos:
        nuevaSol = pd.read_json(pathSoluciones + archivo,orient='index')
        puzzleSol[nuevaSol[0]['puzzleName']] = nuevaSol[0]['shapeData']


    #HASTA AQUI TENEMOS EN PUZZLE SOL CADA PUZZLE CON TODA SU SOLUCION.
    #AHORA VAMOS A PONER COMO VALOR DEL DICT SOLO LAS FORMAS QUE CONTIENE SIN TENER EN CUENTA ROTACIONES, ESCALA...
    puzzleSolTypes = dict()
    for puzzle in puzzleSol.keys():
        listaFormas = []
        for shape in puzzleSol[puzzle]:
            listaFormas.append(shape['shapeType'])
        puzzleSolTypes[puzzle] = listaFormas


    errorList = []
    for user in data['user'].unique():
                #Select rows
                user_events = data[data['user'] == user]
                user_events_na_dropped = user_events.dropna()
                for attempt in user_events_na_dropped['n_attempt'].unique():
                    attempt_events = user_events_na_dropped[user_events_na_dropped['n_attempt'] == attempt]
                    shapeList = []
                    for enum, event in attempt_events.iterrows():
                        if ("create" in event['type'] and "delete" not in event['type']):
                            shapeList.append(event['metadata']['shape_type'])
                            listaSolucion = puzzleSolTypes[event['task_id']]
                            listaSolucion.sort()
                            shapeList.sort()
                            event['shapeTypes_solution'] = listaSolucion
                            event['shapeTypes_student'] = shapeList.copy()
                        elif ("create" in event['type'] and "delete" in event['type']):
                            listaSolucion = puzzleSolTypes[event['task_id']]
                            listaSolucion.sort()
                            shapeList.sort()
                            event['shapeTypes_solution'] = listaSolucion
                            event['shapeTypes_student'] = shapeList.copy()
                        elif("delete" in event['type']):
                            if (event['metadata']['shape_type'] in shapeList):
                                shapeList.remove(event['metadata']['shape_type'])
                            listaSolucion = puzzleSolTypes[event['task_id']]
                            listaSolucion.sort()
                            shapeList.sort()
                            event['shapeTypes_solution'] = listaSolucion
                            event['shapeTypes_student'] = shapeList.copy()
                        elif (event['type'] == 'ws-check_solution'):
                            listaSolucion = puzzleSolTypes[event['task_id']]
                            listaSolucion.sort()
                            shapeList.sort()
                            event['shapeTypes_solution'] = listaSolucion
                            event['shapeTypes_student'] = shapeList.copy()
                            currentList = event['shapeTypes_solution'].copy()
                            differentList = []
                            for shape in event['shapeTypes_student']:
                                if shape in currentList:
                                    currentList.remove(shape)
                                else:
                                    differentList.append(shape) 
                            event['metadata']['shapesIncorrect'] = differentList
                        else:
                            listaSolucion = puzzleSolTypes[event['task_id']]
                            listaSolucion.sort()
                            shapeList.sort()
                            event['shapeTypes_solution'] = listaSolucion
                            event['shapeTypes_student'] = shapeList.copy()
                        errorList.append(event)
E
    errorDf = pd.DataFrame(errorList, columns=['group_id', 'user', 'type', 'n_attempt','task_id', 'shapeTypes_solution', 'shapeTypes_student', 'metadata'])                     

    commonList = []
    for user in errorDf['user'].unique():
        user_events = errorDf[errorDf['user'] == user]
        for puzzle in user_events['task_id'].unique():
            puzzle_events = user_events[user_events['task_id'] == puzzle]
            previousAttempt = False
            changeDict = {}
            for enum, event in puzzle_events.iterrows():
                if event['type'] == 'ws-check_solution':
                    event['changes'] = changeDict
                    changeDict = {}
                    changeDict['created_shapes'] = set()
                    changeDict['deleted_shapes'] = set()
                    changeDict['moved_shapes'] = set()
                    changeDict['scaled_shapes'] = set()
                    changeDict['rotated_shapes'] = set()
                    previousAttempt = True
                else:
                    event['changes'] = {}
                if previousAttempt == True:
                    if ('create' in event['type']):
                        changeDict['created_shapes'].add(event['metadata']['shape_type'])
                    if ('delete' in event['type']):
                        changeDict['deleted_shapes'].add(event['metadata']['shape_type'])
                    if ('rotate' in event['type'] and 'shape' in event['type']):
                        changeDict['rotated_shapes'].add(event['metadata']['shape_type'])
                    if ('scale' in event['type']):
                        changeDict['scaled_shapes'].add(event['metadata']['shape_type'])
                    if ('move' in event['type']):
                        changeDict['moved_shapes'].add(event['metadata']['shape_type'])
                commonList.append(event)
                
    changedDf = pd.DataFrame(commonList, columns=['group_id','user', 'type', 'n_attempt','task_id', 'shapeTypes_solution', 'shapeTypes_student', 'metadata', 'changes'])                     
    
    finalList = []
    checks = changedDf[changedDf['type'] == 'ws-check_solution']
    for enum,event in checks.iterrows():
        event['correct'] = event['metadata']['correct']
        event['shapes_Incorrect'] = event['metadata']['shapesIncorrect']
        finalList.append(event)

    changedDf = pd.DataFrame(finalList, columns=['group_id', 'user', 'type', 'n_attempt','task_id','shapeTypes_solution', 'shapeTypes_student', 'correct', 'shapes_Incorrect', 'changes'])
    changedDf
    
    return changedDf

In [9]:
errors = commonErrors(dataFrame = eventsToUse)
errors

,group_id,user,type,n_attempt,task_id,shapeTypes_solution,shapeTypes_student,correct,shapes_Incorrect,changes
410417,cb71040b5bd1341a34afc24961536ebd,56ccce25ead834182d605eff319bfa2c,ws-check_solution,1,Sugar Cones,"[5, 5]","[5, 5, 5, 5, 5]",False,"[5, 5, 5]",{}
4276,cb71040b5bd1341a34afc24961536ebd,56ccce25ead834182d605eff319bfa2c,ws-check_solution,16,Sugar Cones,"[5, 5]","[5, 5]",True,[],"{'created_shapes': {5}, 'deleted_shapes': {5},..."
423407,cb71040b5bd1341a34afc24961536ebd,56ccce25ead834182d605eff319bfa2c,ws-check_solution,7,45-Degree Rotations,"[3, 3, 3, 3, 6]","[3, 3, 3, 3, 6]",True,[],{}
423478,cb71040b5bd1341a34afc24961536ebd,56ccce25ead834182d605eff319bfa2c,ws-check_solution,8,Pyramids are Strange,"[2, 2, 2, 2]","[2, 2, 2, 2]",False,[],{}
423485,cb71040b5bd1341a34afc24961536ebd,56ccce25ead834182d605eff319bfa2c,ws-check_solution,8,Pyramids are Strange,"[2, 2, 2, 2]","[2, 2, 2, 2]",True,[],"{'created_shapes': {}, 'deleted_shapes': {}, '..."
...,...,...,...,...,...,...,...,...,...,...
390320,e21640b4aea9349ad77d86d6017cb061,7e95e0d124f5d7852af4b05082db7b1c,ws-check_solution,5,45-Degree Rotations,"[3, 3, 3, 3, 6]","[3, 3, 3, 3, 6]",False,[],{}
390920,e21640b4aea9349ad77d86d6017cb061,7e95e0d124f5d7852af4b05082db7b1c,ws-check_solution,5,45-Degree Rotations,"[3, 3, 3, 3, 6]","[3, 3, 3, 3, 6]",True,[],"{'created_shapes': {}, 'deleted_shapes': {}, '..."
391830,e21640b4aea9349ad77d86d6017cb061,7e95e0d124f5d7852af4b05082db7b1c,ws-check_solution,6,Pyramids are Strange,"[2, 2, 2, 2]","[2, 2, 2, 2]",True,[],{}
395768,e21640b4aea9349ad77d86d6017cb061,7e95e0d124f5d7852af4b05082db7b1c,ws-check_solution,10,Sugar Cones,"[5, 5]","[5, 5]",False,[],{}


In [10]:
def commonErrorsByPuzzle(dataFrame):
    
    pathSoluciones = "/Users/manuelgomezmoratilla/Downloads/Fall19studylevels-final (1)/"
    changedDf = dataFrame #Como parametro

    listaArchivos = []
    for i in range(1,10):
            listaArchivos.append("public" + str(i)+".json")
            listaArchivos.append("annie" + str(i)+".json")

    puzzleSol = dict()
    for archivo in listaArchivos:
        nuevaSol = pd.read_json(pathSoluciones + archivo,orient='index')
        puzzleSol[nuevaSol[0]['puzzleName']] = nuevaSol[0]['shapeData']


    #HASTA AQUI TENEMOS EN PUZZLE SOL CADA PUZZLE CON TODA SU SOLUCION.
    #AHORA VAMOS A PONER COMO VALOR DEL DICT SOLO LAS FORMAS QUE CONTIENE SIN TENER EN CUENTA ROTACIONES, ESCALA...
    puzzleSolTypes = dict()
    for puzzle in puzzleSol.keys():
        listaFormas = []
        for shape in puzzleSol[puzzle]:
            listaFormas.append(shape['shapeType'])
        puzzleSolTypes[puzzle] = listaFormas
    
    errorsDict = dict()
    for group in changedDf['group_id'].unique():
        group_events = changedDf[changedDf['group_id'] == group]
        if group not in errorsDict.keys():
            errorsDict[group] = dict()
        for puzzle in group_events['task_id'].unique():
            puzzleEvents = group_events[group_events['task_id'] == puzzle]
            for enum, event in puzzleEvents.iterrows():
                if puzzle not in errorsDict[group].keys():
                    errorsDict[group][puzzle] = dict()
                    #errorsDict[group][puzzle]['created_shapes'] = dict()
                    #errorsDict[group][puzzle]['deleted_shapes'] = dict()
                    #errorsDict[group][puzzle]['moved_shapes'] = dict()
                    #errorsDict[group][puzzle]['scaled_shapes'] = dict()
                    #errorsDict[group][puzzle]['rotated_shapes'] = dict()
                if len(event['changes']) > 0:
                    for key in event['changes'].keys():
                        setShapes = event['changes'][key]
                        for shape in setShapes:
                            if key not in errorsDict[group][puzzle].keys():
                                errorsDict[group][puzzle][key] = dict()
                            if shape not in errorsDict[group][puzzle][key].keys():
                                errorsDict[group][puzzle][key][shape] = 1
                            else : 
                                errorsDict[group][puzzle][key][shape] += 1 P
                                
    for group in errorsDict.keys():
        for puzzle in errorsDict[group].keys():
            total = 0
            for event_type in errorsDict[group][puzzle].keys():
                for shape_type in errorsDict[group][puzzle][event_type].keys():
                    total += errorsDict[group][puzzle][event_type][shape_type]
            errorsDict[group][puzzle]['total'] = total

    for group in errorsDict.keys():
        for puzzle in errorsDict[group].keys():
            for event_type in errorsDict[group][puzzle].keys():
                if event_type == 'total':
                    continue
                for shape_type in errorsDict[group][puzzle][event_type].keys():
                    errorsDict[group][puzzle][event_type][shape_type] = (errorsDict[group][puzzle][event_type][shape_type] / errorsDict[group][puzzle]['total']) * 100                        

    shapes_map = {1 : 'cube', 2 : 'pyramid', 3 : 'ramp', 4 : 'cylinder', 5 : 'cone', 6: 'sphere'} 
    newList = []
    for group in errorsDict.keys():
        for puzzle in errorsDict[group].keys():
            newEvent = pd.Series(dtype='object')
            newEvent['group_id'] = group
            newEvent['task_id'] = puzzle
            #Para R
            #newEvent['master_solution'] = dict()
            newEvent['master_solution'] = list()
            #i = 0
            for shape in puzzleSolTypes[puzzle]:
                #newEvent['master_solution']['master' + str(i)] = (shapes_map[shape])
                #i +=1
                newEvent['master_solution'].append(shapes_map[shape])
            newEvent['common_Errors'] = dict()
            for key in errorsDict[group][puzzle].keys():
                if key == 'total' :
                    continue
                for typeShape in errorsDict[group][puzzle][key]:
                    if errorsDict[group][puzzle][key][typeShape] > 15:
                        if key not in newEvent['common_Errors'].keys():
                            newEvent['common_Errors'][key] = dict()
                        newEvent['common_Errors'][key][shapes_map[typeShape]] = round(errorsDict[group][puzzle][key][typeShape],2)
            newList.append(newEvent)

    resumeDf = pd.DataFrame(newList, columns=['group_id', 'task_id', 'master_solution', 'common_Errors'])

    return resumeDf


In [11]:
pd.set_option('display.max_colwidth', 150)
common = commonErrorsByPuzzle(dataFrame = errors)

In [12]:
common

,group_id,task_id,master_solution,common_Errors
0,cb71040b5bd1341a34afc24961536ebd,Sugar Cones,"[cone, cone]","{'deleted_shapes': {'cone': 25.0}, 'moved_shapes': {'cone': 37.5}, 'rotated_shapes': {'cone': 25.0}}"
1,cb71040b5bd1341a34afc24961536ebd,45-Degree Rotations,"[ramp, ramp, ramp, ramp, sphere]","{'rotated_shapes': {'ramp': 50.0}, 'moved_shapes': {'ramp': 50.0}}"
2,cb71040b5bd1341a34afc24961536ebd,Pyramids are Strange,"[pyramid, pyramid, pyramid, pyramid]","{'moved_shapes': {'pyramid': 24.32}, 'rotated_shapes': {'pyramid': 59.46}}"
3,cb71040b5bd1341a34afc24961536ebd,Object Limits,[cube],"{'scaled_shapes': {'cube': 54.84}, 'moved_shapes': {'cube': 22.58}}"
4,cb71040b5bd1341a34afc24961536ebd,Stranger Shapes,"[pyramid, pyramid]","{'moved_shapes': {'pyramid': 42.42}, 'rotated_shapes': {'pyramid': 33.33}, 'scaled_shapes': {'pyramid': 18.18}}"
...,...,...,...,...
89,e21640b4aea9349ad77d86d6017cb061,45-Degree Rotations,"[ramp, ramp, ramp, ramp, sphere]","{'rotated_shapes': {'ramp': 75.0}, 'moved_shapes': {'ramp': 25.0}}"
90,e21640b4aea9349ad77d86d6017cb061,Pyramids are Strange,"[pyramid, pyramid, pyramid, pyramid]","{'rotated_shapes': {'pyramid': 41.18}, 'scaled_shapes': {'pyramid': 21.57}, 'moved_shapes': {'pyramid': 17.65}}"
91,e21640b4aea9349ad77d86d6017cb061,Sugar Cones,"[cone, cone]","{'moved_shapes': {'cone': 80.0}, 'rotated_shapes': {'cone': 20.0}}"
92,e21640b4aea9349ad77d86d6017cb061,Boxes Obscure Spheres,"[sphere, cylinder, cylinder, cube]",{}


In [66]:
def showCommonErrors(group, dataFrameErrors) :
    mapToString = {'deleted_shapes' : 'Common deleted shapes', 'created_shapes' : 'Common created shapes', 'scaled_shapes' : 'Common scaled shapes', 'rotated_shapes' : 'Common rotated shapes', 'moved_shapes' : 'Common moved shapes'}
    stringErrors = ""
    groupDf = dataFrameErrors[dataFrameErrors['group_id'] == group]
    for puzzle in groupDf['task_id'].unique():
        puzzle_Event = groupDf[groupDf['task_id'] == puzzle]
        for enum, event in puzzle_Event.iterrows():
            if len(event['common_Errors']) > 0:
                stringErrors = stringErrors + puzzle + ":\n"
                stringErrors += "Master solution - "
                for shape in event['master_solution']:
                    stringErrors += shape + " "
                stringErrors += '\n'
                for key in event['common_Errors'].keys():
                    stringErrors += mapToString[key] + "- "
                    for typeShape in event['common_Errors'][key]:
                        stringErrors += typeShape + ": " +  str(event['common_Errors'][key][typeShape]) + "% "
                    stringErrors += "\n"
                stringErrors += "\n\n"
                
            
        
    print(stringErrors)
    
        
    


In [67]:
showCommonErrors(group = 'cb71040b5bd1341a34afc24961536ebd', dataFrameErrors = common)

Sugar Cones:
Master solution - cone cone 
Common deleted shapes- cone: 25.0% 
Common moved shapes- cone: 37.5% 
Common rotated shapes- cone: 25.0% 


45-Degree Rotations:
Master solution - ramp ramp ramp ramp sphere 
Common rotated shapes- ramp: 50.0% 
Common moved shapes- ramp: 50.0% 


Pyramids are Strange:
Master solution - pyramid pyramid pyramid pyramid 
Common moved shapes- pyramid: 24.32% 
Common rotated shapes- pyramid: 59.46% 


Object Limits:
Master solution - cube 
Common scaled shapes- cube: 54.84% 
Common moved shapes- cube: 22.58% 


Stranger Shapes:
Master solution - pyramid pyramid 
Common moved shapes- pyramid: 42.42% 
Common rotated shapes- pyramid: 33.33% 
Common scaled shapes- pyramid: 18.18% 


Tall and Small:
Master solution - cube sphere 
Common moved shapes- sphere: 37.5% cube: 28.12% 
Common scaled shapes- cube: 15.62% 


Ramp Up and Can It:
Master solution - cylinder ramp 
Common moved shapes- cylinder: 66.67% ramp: 33.33% 


Angled Silhouette:
Master solution